<a href="https://colab.research.google.com/github/andersonfurtado/AI4WEBDEV/blob/main/MovieLens_SQLAlchemy_Database_Creation.ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Install the necessary libraries
!pip install sqlalchemy pandas openpyxl scikit-surprise

# Import the libraries
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.orm import declarative_base, sessionmaker
from surprise import Dataset, Reader, SVD
import urllib.request
import zipfile
import os

# Download and unzip the MovieLens dataset
url = 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
urllib.request.urlretrieve(url, 'ml-latest-small.zip')

with zipfile.ZipFile('ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall()

# Full path to the files
movies_file = os.path.join('ml-latest-small', 'movies.csv')
ratings_file = os.path.join('ml-latest-small', 'ratings.csv')

# Load the data
movies = pd.read_csv(movies_file)
ratings = pd.read_csv(ratings_file)

# Define the SQLAlchemy models
Base = declarative_base()

class Movie(Base):
    __tablename__ = 'movies'
    movieId = Column(Integer, primary_key=True)
    title = Column(String)
    genres = Column(String)

class Rating(Base):
    __tablename__ = 'ratings'
    userId = Column(Integer, primary_key=True)
    movieId = Column(Integer, primary_key=True)
    rating = Column(Float)
    timestamp = Column(Integer, primary_key=True)

# Create the SQLite database and tables
engine = create_engine('sqlite:///movielens.db')
Base.metadata.create_all(engine)

# Create a session
Session = sessionmaker(bind=engine)
session = Session()

# Insert data into the movie table
movies_data = movies.to_dict(orient='records')
existing_movies = {movie.movieId for movie in session.query(Movie.movieId).all()}
new_movies = [Movie(**data) for data in movies_data if data['movieId'] not in existing_movies]
session.bulk_save_objects(new_movies)

# Insert data into the ratings table
ratings_data = ratings.to_dict(orient='records')
existing_ratings = {(rating.userId, rating.movieId, rating.timestamp) for rating in session.query(Rating.userId, Rating.movieId, Rating.timestamp).all()}
new_ratings = [Rating(**data) for data in ratings_data if (data['userId'], data['movieId'], data['timestamp']) not in existing_ratings]
session.bulk_save_objects(new_ratings)

# Commit session
session.commit()

# Prepare the data for Surprise library
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Train the SVD model
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

# Function to get movie recommendations from the database
def get_movie_recommendations_from_db(movie_title, num_recommendations=5):
    movie = session.query(Movie).filter_by(title=movie_title).first()
    if not movie:
        return []

    movie_id = movie.movieId
    users_who_rated_movie = session.query(Rating.userId).filter_by(movieId=movie_id).distinct().all()
    users_who_rated_movie = [u[0] for u in users_who_rated_movie]

    other_movie_ids = session.query(Rating.movieId).filter(Rating.userId.in_(users_who_rated_movie)).distinct().all()
    other_movie_ids = [m[0] for m in other_movie_ids]

    predicted_ratings = []
    for movie_id in other_movie_ids:
        predicted_rating = algo.predict(uid=0, iid=movie_id).est
        predicted_ratings.append((movie_id, predicted_rating))

    predicted_ratings.sort(key=lambda x: x[1], reverse=True)
    top_n_movies = [movie_id for movie_id, rating in predicted_ratings[:num_recommendations]]
    recommended_movies = session.query(Movie.title).filter(Movie.movieId.in_(top_n_movies)).all()
    return [m[0] for m in recommended_movies]

# Example of use
print(get_movie_recommendations_from_db('Toy Story (1995)'))


['Shawshank Redemption, The (1994)', 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)', 'Philadelphia Story, The (1940)', 'Rear Window (1954)', 'Lawrence of Arabia (1962)']
